In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
import seaborn as sns
import time
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
import gc
from sklearn.model_selection import cross_val_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from hyperopt import STATUS_OK, tpe, hp, Trials, fmin
import warnings
warnings.filterwarnings('ignore')

In [2]:
X_all_features = pd.read_csv('./new_features/X_train_all_features.csv')

In [4]:
X_all_features2 = pd.read_csv('./new_features/X_train_all_features2.csv')

In [6]:
X_all_features2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
X1 = pd.concat([X_all_features, X_all_features2], axis=0, ignore_index=True)

In [9]:
X_new_features = pd.read_csv('./new_features/X_train_new_features.csv')

In [12]:
X_new_features2 = pd.read_csv('./new_features/X_train_new_features2.csv')

In [16]:
X2 = pd.concat([X_new_features, X_new_features2], axis=0, ignore_index=True)

In [17]:
del(X_all_features, X_all_features2, X_new_features, X_new_features2)
import gc
gc.collect()

7

In [20]:
X = pd.concat([X1, X2], axis=1)

In [22]:
drop_columns = ['classic_sta_lta5_mean', 'classic_sta_lta7_mean', 'den_mean_change_rate_first_10000', 'den_mean_change_rate_last_10000']

In [24]:
X.drop(drop_columns, axis=1, inplace=True)

In [25]:
X.shape

(6292, 1299)

In [26]:
X.to_csv('./new_features/X.csv', index=False, header=True)

In [29]:
X_test_all_features = pd.read_csv('./new_features/X_test_all_features.csv', index_col='seg_id')

In [30]:
X_test_new_features = pd.read_csv('./new_features/X_test_new_features.csv', index_col='seg_id')

In [34]:
X_test_all_features.head()

,mean,std,max,min,mean_change_abs,mean_change_rate,abs_max,mean_first_50000,mean_last_50000,mean_first_10000,...,den_std_roll_10000_mean,den_std_roll_10000_std,den_std_roll_10000_max,den_std_roll_10000_min,den_std_roll_10000_q01,den_std_roll_10000_q05,den_std_roll_10000_median,den_std_roll_10000_q95,den_std_roll_10000_q99,den_std_roll_10000_av_change
seg_id,,,,,,,,,,,,,,,,,,,,,
seg_00030f,4.491780000000000,4.893689687028084,115.0,-75.0,0.000026666844446,75044.983541275520111,115.0,4.46644,4.49598,4.3842,...,2.760738879234107,1.767032295683657,6.967250708302509,0.164405001196263,0.412846000585372,0.826215392499982,2.507467554558998,6.888663857287961,6.945255249711700,0.000005375311823
seg_0012b5,4.171153333333334,5.922839443206501,152.0,-140.0,-0.000013333422223,74949.685652956031845,152.0,4.01786,4.24444,4.0635,...,3.895347848312298,2.597804433881185,9.684650699455405,0.621692447357012,0.621692447357012,1.005311049555419,2.724245903661253,8.196541548667495,9.670105862920671,0.000002960198122
seg_00184e,4.610260000000000,6.946990077499338,248.0,-193.0,-0.000020000133334,74997.632054261513986,248.0,4.55518,4.55380,4.2452,...,4.161058344029042,4.262120691132202,18.718209207638580,0.505298887790425,0.505298887790425,0.666166329089699,2.495974326304224,15.550837465973933,18.710928539918228,-0.000011807302605
seg_003339,4.531473333333333,4.114146602958790,85.0,-93.0,0.000046666977780,74997.954642264914582,93.0,4.49052,4.48922,4.3834,...,2.188771447927612,1.763563346630023,7.156822811026635,0.000000091560300,0.000000357584224,0.000000357584224,1.819209140846672,7.085477479461633,7.156822811026635,-0.000016477429210
seg_0042cc,4.128340000000001,5.797163636219714,177.0,-147.0,-0.000006666711111,75075.108146992672118,177.0,4.23020,4.10524,4.4902,...,3.511429543748389,3.108405193585451,12.939748054498009,0.203877298025122,0.305003526711859,0.622230050386334,2.664548667326618,12.652994419268135,12.937472170513461,0.000012995710112


In [35]:
X_test_new_features.head()

,FFT_Mag_01q0,FFT_Mag_10q0,FFT_Mag_90q0,FFT_Mag_99q0,FFT_Mag_mean0,FFT_Mag_std0,FFT_Mag_max0,FFT_Phz_mean0,FFT_Phz_std0,FFT_Mag_01q2500,...,MA_700MA_BB_high_mean_9,MA_700MA_BB_low_mean_9,MA_400MA_std_mean_9,MA_400MA_BB_high_mean_9,MA_400MA_BB_low_mean_9,MA_1000MA_std_mean_9,iqr_9,q999_9,q001_9,ave10_9
seg_id,,,,,,,,,,,,,,,,,,,,,
seg_00030f,133.370417601944098,379.989767494343198,2099.323960739667200,3454.405454203950740,1158.604141783657496,768.254345585969986,11238.366663567248906,-0.012250133888193,0.900690532059439,320.841247416905162,...,4.004238824668860,-4.004238782614506,2.002314287920792,4.004628596868762,-4.004628554814406,2.002027236819957,2.698087496082402,6.211311107283239,-6.258732613455558,-0.000632433816681
seg_0012b5,101.463063522838922,403.043357421815472,2706.287890365305884,4659.971868325285868,1376.586870665243623,990.128585620922649,10991.826903007111468,-0.010391611306858,0.915303606569094,401.527060926127035,...,4.011673947798466,-4.011673978799556,2.005447188981329,4.010894362462113,-4.010894393463204,2.006163205382212,2.696517374462489,6.278556245239137,-6.302027815937049,-0.000758933791516
seg_00184e,151.454194190503244,425.323133816631866,2487.224447950288777,4043.233209372576766,1305.790884233702627,882.537006155851259,11014.068310970902530,-0.019541614662970,0.918201650791437,409.000861237515437,...,4.028395207703538,-4.028395234313781,2.014105681811720,4.028211350318319,-4.028211376928562,2.014495411351019,2.720088566754943,6.304279480025097,-6.274640871253313,-0.002937300096151
seg_003339,105.571239563327353,349.567502009401096,1821.013388192713592,2892.954016195356417,1038.729254882910027,661.926156279097086,9852.436660269195272,-0.009640276819388,0.904850116703317,205.126928206764973,...,3.986394159992610,-3.986394142710948,1.993211920690688,3.986423850022208,-3.986423832740545,1.993194120792260,2.677482804697490,6.179592910946313,-6.213242493464440,-0.001001348414325
seg_0042cc,131.767621584348063,388.357916763667617,2239.528336154414319,3588.816127630382198,1213.050949849615336,842.284316482899271,13054.367373216007763,0.006268816327690,0.907299329094731,330.294872339379594,...,3.989340105461091,-3.989340138140896,1.994490132416861,3.988980248493819,-3.988980281173624,1.994818502563433,2.675930537338367,6.250093516957571,-6.236563248697144,-0.001628469728399


In [37]:
X_test = pd.concat([X_test_all_features, X_test_new_features], axis=1)

In [38]:
X_test.drop(drop_columns, axis=1, inplace=True)

In [40]:
X_test.to_csv('./new_features/X_test.csv', index=True, header=True)

In [2]:
y = pd.read_csv('./new_features/y.csv')

In [4]:
y2 = pd.read_csv('./new_features/y2.csv')

In [7]:
new_y = pd.concat([y, y2], axis=0, ignore_index=True)

In [8]:
y.head()

,time_to_failure
0,1.430797219276428
1,1.391498923301697
2,1.353196144104004
3,1.313797831535339
4,1.274399518966675


In [9]:
new_y.head()

,time_to_failure
0,1.430797219276428
1,1.391498923301697
2,1.353196144104004
3,1.313797831535339
4,1.274399518966675


In [10]:
new_y.to_csv('./new_features/new_y.csv', index=False, header=True)